In [0]:
#install.packages("pls")
#install.packages("caret")
install.packages("FNN")

In [0]:
#library(pls)
#library(caret)
library(FNN)

In [17]:
LA_Housing <- read.csv("Final_Housing_Updated3.csv")
colnames(LA_Housing)
LA_Housing$Date <- as.Date(LA_Housing$Date)
LA_Housing <- LA_Housing[LA_Housing$PROPERTY.TYPE == "Single Family Residential",]

LA_Housing$LONG_LAT <- LA_Housing$LATITUDE * LA_Housing$LONGITUDE
LA_Housing$SQ_LOT <- as.double(LA_Housing$SQUARE.FEET) * as.double(LA_Housing$LOT.SIZE)
feb_first <- as.Date("2020-02-01")
train <- LA_Housing[LA_Housing$Date <= feb_first,]
test <- LA_Housing[LA_Housing$Date > feb_first,]


[1] "ZIP"              "Date"             "PROPERTY.TYPE"    "ADDRESS"         
 [5] "CITY"             "STATE"            "PRICE"            "BEDS"            
 [9] "BATHS"            "SQUARE.FEET"      "LOT.SIZE"         "YEAR.BUILT"      
[13] "DAYS.ON.MARKET"   "LATITUDE"         "LONGITUDE"        "Age"             
[17] "ORIGINAL_PRC"     "AvgTotalIncome"   "AvgRealEstateTax" "AvgPropertyTax"  
[21] "AvgMortgageInt"   "PrcRank"          "PrcRank2"         "PROPERTY.TYPE.N" 
[25] "numCrimes"        "ID"

In [7]:
head(train)
print(nrow(train))

,ZIP,Date,PROPERTY.TYPE,ADDRESS,CITY,STATE,PRICE,BEDS,BATHS,SQUARE.FEET,⋯,AvgRealEstateTax,AvgPropertyTax,AvgMortgageInt,PrcRank,PrcRank2,PROPERTY.TYPE.N,numCrimes,ID,LONG_LAT,SQ_LOT
,<int>,<date>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<dbl>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,90001,2018-04-06,Single Family Residential,1567 E Gage Ave,Los Angeles,CA,450800,3,2.5,1412,⋯,0.0109717,0.9061453,9.211386,1,1,4,0,1,-4018.265,2603728
2,90001,2017-06-20,Single Family Residential,8210 Lou Dillon Ave,Los Angeles,CA,363500,2,1.0,844,⋯,0.0109717,0.9061453,9.211386,1,1,4,0,2,-4015.719,3903500
3,90001,2018-09-26,Single Family Residential,1667 E 84th St,Los Angeles,CA,311300,2,1.0,962,⋯,0.0109717,0.9061453,9.211386,1,1,4,0,3,-4015.852,4818658
5,90001,2019-03-07,Single Family Residential,7672 Parmelee Ave,Los Angeles,CA,324500,2,1.0,944,⋯,0.0109717,0.9061453,9.211386,1,1,4,0,5,-4016.944,3826032
6,90001,2018-11-08,Single Family Residential,1155 E 75th St,Los Angeles,CA,362000,2,1.0,762,⋯,0.0109717,0.9061453,9.211386,1,1,4,0,6,-4017.402,3559302
8,90001,2018-09-27,Single Family Residential,7510 Parmelee Ave,Los Angeles,CA,400500,2,1.0,804,⋯,0.0109717,0.9061453,9.211386,1,1,4,0,8,-4017.206,2027688


[1] 24210


In [0]:
pcr.fit = pcr(PRICE ~ ZIP + BEDS + BATHS + SQUARE.FEET + LOT.SIZE + Age + LATITUDE + LONGITUDE + LATITUDE*LONGITUDE, data = train, scale = TRUE, validation = "LOO")
summary(pcr.fit)

Data: 	X dimension: 30867 9 
	Y dimension: 30867 1
Fit method: svdpc
Number of components considered: 9

VALIDATION: RMSEP
Cross-validated using 30867 leave-one-out segments.
       (Intercept)  1 comps  2 comps  3 comps  4 comps  5 comps  6 comps
CV         1388222  1384250  1180340  1180053  1154019  1133033  1100575
adjCV      1388222  1384250  1180340  1180052  1154019  1133033  1100576
       7 comps  8 comps  9 comps
CV      907561   908292   880930
adjCV   907561   908292   880930

TRAINING: % variance explained
       1 comps  2 comps  3 comps  4 comps  5 comps  6 comps  7 comps  8 comps
X      34.9100    59.64    70.76    80.89    87.97    93.50    97.34   100.00
PRICE   0.5776    28.32    28.36    31.44    33.92    39.17    57.41    57.42
       9 comps
X       100.00
PRICE    59.94


In [0]:
pls.fit = plsr = pcr(PRICE ~ ZIP + BEDS + BATHS + SQUARE.FEET + LOT.SIZE + Age + LATITUDE + LONGITUDE + LATITUDE*LONGITUDE, data = train, scale = TRUE, validation = "LOO")
summary(pls.fit)

In [0]:
knnRegression <- train(PRICE ~ ZIP + PROPERTY.TYPE + BEDS + BATHS + SQUARE.FEET + LOT.SIZE + Age + LATITUDE + LONGITUDE, data = train, method = "knn", trControl = trainControl("LOOCV"), preProcess = c("center","scale"), tuneLength = 20)
prediction <- predict(knnRegression,train)
RMSE(prediction,train$PRICE)
plot(knnRegression)

In [0]:
rmseFun <- function(predicted,y){
  return(sqrt(mean((predicted - y)^2)))
}
numK <- 2:15
rmseList = rep(list(numeric(nrow(train))),length(numK))

knnDF <- as.data.frame(scale(train[c("ZIP","BEDS","BATHS","SQUARE.FEET","SQ_LOT","Age","LATITUDE","LONGITUDE","LOT.SIZE","LONG_LAT", "numCrimes")]))


In [0]:
#LOOCV
for(i in 1:length(numK)) {
  for(k in 1:nrow(train)) {
    model <- knn.reg(train = knnDF[-k,],
                     test = knnDF[k,],
                     y = train[-k,]$PRICE, k = numK[i])
    rmseList[[i]][k] <- rmseFun(model$pred,train[k,]$PRICE)
  }
}

In [22]:
sum(rmseList[[1]] != 0)

[1] 198